In [1]:
import tabulate
import operator
import os
import pandas as pd
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents.agent_types import AgentType
from dotenv import load_dotenv, dotenv_values 

load_dotenv()

True

In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] ="multi-agent"
os.environ['OPENAI_API_KEY'] = openai_api_key

In [11]:
file_path= 'data.csv'

csv_agent= create_csv_agent(
        ChatOpenAI(temperature=0, model="gpt-4"),
        file_path, 
        verbose=True,
        stop=["\nObservation:"],
        agent_type=AgentType.OPENAI_FUNCTIONS,
        handle_parsing_errors=True
    )
    
resut = csv_agent.run("give me some graphs for the country of france which will help me understand the business over there, store all the graphs with insights in a pdf and save it")

c:\SmartCart_MBA\streamlit_app\venv\lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:242: UserWarning: Received additional kwargs {'stop': ['\nObservation:'], 'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(




> Entering new AgentExecutor chain...
To generate graphs and insights for the business in France, we can follow these steps:

1. Filter the dataframe for transactions from France.
2. Generate various plots such as:
   - A bar plot of the most sold products.
   - A line plot of sales over time.
   - A pie chart of sales distribution among different customers.
3. Save these plots and insights into a PDF.

However, I'm an AI language model and currently, I don't have the capability to directly generate plots or PDFs. But I can provide you with the Python code that you can use to perform these tasks.

Here's an example of how you can do it using pandas, matplotlib, and a library called matplotlib.backends.backend_pdf for creating the PDF:

```python
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Filter the dataframe for transactions from France
df_france = df[df['Country'] == 'France']

# Create a PDF object
pdf_pages = PdfPage

In [12]:
print(resut)

To generate graphs and insights for the business in France, we can follow these steps:

1. Filter the dataframe for transactions from France.
2. Generate various plots such as:
   - A bar plot of the most sold products.
   - A line plot of sales over time.
   - A pie chart of sales distribution among different customers.
3. Save these plots and insights into a PDF.

However, I'm an AI language model and currently, I don't have the capability to directly generate plots or PDFs. But I can provide you with the Python code that you can use to perform these tasks.

Here's an example of how you can do it using pandas, matplotlib, and a library called matplotlib.backends.backend_pdf for creating the PDF:

```python
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Filter the dataframe for transactions from France
df_france = df[df['Country'] == 'France']

# Create a PDF object
pdf_pages = PdfPages('france_sales_report.pdf')

# Generate

In [15]:
import re
pattern = r"python```"

matched = re.search(pattern, resut, re.DOTALL)

if matched:
    extracted_code = matched.group(1).strip()
    print(extracted_code)
else:
    print("No Python code block found.")

No Python code block found.


In [25]:
print(resut.split("python")[1].split('```')[0])


import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Filter the dataframe for transactions from France
df_france = df[df['Country'] == 'France']

# Create a PDF object
pdf_pages = PdfPages('france_sales_report.pdf')

# Generate a bar plot of the most sold products
plt.figure(figsize=(10,6))
df_france['ProductName'].value_counts().head(10).plot(kind='bar')
plt.title('Top 10 Sold Products in France')
plt.xlabel('Product Name')
plt.ylabel('Quantity Sold')
pdf_pages.savefig()  # saves the current figure into a pdf page
plt.clf()

# Generate a line plot of sales over time
df_france['Date'] = pd.to_datetime(df_france['Date'])
df_france.groupby('Date')['Quantity'].sum().plot()
plt.title('Sales Over Time in France')
plt.xlabel('Date')
plt.ylabel('Quantity Sold')
pdf_pages.savefig()  # saves the current figure into a pdf page
plt.clf()

# Generate a pie chart of sales distribution among different customers
df_france['CustomerNo'].value_cou

In [13]:
type(resut)

str

In [11]:
csv_agent.run("Which product generates more revenue?")




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Revenue'] = df['Price'] * df['Quantity']"}`
responded: To find out which product generates more revenue, we need to create a new column in the dataframe that represents the total revenue for each transaction. This can be calculated by multiplying the Price and Quantity columns. Then, we can group by the ProductName and sum the revenue, sorting in descending order to find the product that generates the most revenue.

Let's do this in steps. First, let's create the new column 'Revenue'.


Invoking: `python_repl_ast` with `{'query': "df.groupby('ProductName')['Revenue'].sum().sort_values(ascending=False).head(1)"}`
responded: Now that we have the 'Revenue' column, let's group by 'ProductName' and sum the 'Revenue', then sort in descending order to find the product that generates the most revenue.

ProductName
Popcorn Holder    583286.51
Name: Revenue, dtype: float64The product that generates the most

"The product that generates the most revenue is the 'Popcorn Holder' with a total revenue of 583286.51."

In [28]:
question = "Give me a detailed report for france"

propmt = """ Generate Python code that creates a comprehensive series of graphs to analyze business data for country. Also include the code for loading the data (data.csv)
The visualization should potentially include, but not be limited to, a bar plot of the top-selling products, a line plot showing sales trends over time, and a pie chart detailing the sales distribution among top customers. 
Additionally, consider including other relevant analyses such as sales by category, regional sales distribution, and seasonality effects on sales, if the data permits. 

The graphs should be stored in a PDF named 'sales_report.pdf'. Utilize pandas for data handling, matplotlib for plotting, and the matplotlib.backends.backend_pdf module for generating the PDF file. 
The code should be well-commented to explain the purpose of each graph and how it contributes to understanding the business climate in `country_name`. Include brief insights or interpretations for each plot to highlight the key takeaways and business implications derived from the data analysis.
"""  + question

In [29]:
result = csv_agent.run(propmt)



> Entering new AgentExecutor chain...
Here is a Python code snippet that loads the data from a CSV file, filters it for France, and generates a series of graphs to analyze the business data. The graphs are then saved in a PDF file named 'France_sales_report.pdf'. 

```python
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Load the data
df = pd.read_csv('data.csv')

# Filter the data for France
df_france = df[df['Country'] == 'France']

# Create a PDF for the plots
pdf_pages = PdfPages('France_sales_report.pdf')

# Create a bar plot of the top-selling products
top_products = df_france.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).head(10)
top_products.plot(kind='bar', title='Top 10 Selling Products in France')
plt.xlabel('Product Name')
plt.ylabel('Quantity Sold')
plt.tight_layout()
pdf_pages.savefig()
plt.clf()

# Create a line plot showing sales trends over time
df_france

In [31]:
exec(result.split("python")[1].split('```')[0])

<string>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<Figure size 460.8x345.6 with 0 Axes>

In [16]:
output = csv_agent("How many rows?")

c:\SmartCart_MBA\streamlit_app\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


536350The dataframe has 536,350 rows.

> Finished chain.


In [19]:
output['output']

'The dataframe has 536,350 rows.'

## Lets build a fastapi for the above functionality

In [20]:
# from typing import Union

# from fastapi import FastAPI
# from pydantic import BaseModel

# class InputQuery(BaseModel):
#     query: str

# app = FastAPI()

# @app.put("/csv_agent/")
# def csv_agent_api(query: InputQuery):
#     output = csv_agent(query.query)
#     return {'Query': query.query, 'Output': output['output']}


## Refer csv_agent_api.py file